In [41]:
# Initial Imports
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from pathlib import Path
import regex as re
import nltk
from datetime import date
import twint
import nest_asyncio
nest_asyncio.apply()
nltk.download('stopwords')

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevinwalsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
# Initialize set of inputs
# List of Stocks
stocks = ['microsoft','sony','pfizer','regeneron','moderna']

# Set up Date range
import datetime
from dateutil.relativedelta import relativedelta

today = datetime.date.today()
end_time = today.strftime('%Y-%m-%d %H:%M:%S')
start_time = today - relativedelta(years=1)
start_time = start_time.strftime('%Y-%m-%d %H:%M:%S')
print(start_time)

2020-01-19 00:00:00


In [43]:
# Define empty DataFrames
stock_df = pd.DataFrame()
df = pd.DataFrame()

In [44]:
# Define Data_Crawlers Function
def data_crawlers(stock):
    c = twint.Config()
    c.Search = stock
    c.Hide_output = True
    c.Limit = 1000 
    #c.Store_csv = True
    c.Since = start_time
    c.Until = end_time
    #c.Output = f"{stock}.csv"
    c.Pandas = True
    c.Pandas_au = True
    twint.run.Search(c)
    df.index=idx_microsoft=pd.MultiIndex.from_tuples([tuple([stock, i]) for i in range(len(df.index))])
    return df


In [45]:
df_test = data_crawlers(stocks[0])
df_test.head()

TypeError: Cannot infer number of levels from empty list

In [31]:
# Deploy data_crawlers function on stocks list
for i in stocks: 
    data_crawlers(i)
    df = twint.storage.panda.Tweets_df

    stock_df = stock_df.append(df)

In [32]:
# Visualize the Stock DataFrame
stock_df = stock_df.reset_index(drop=True)
stock_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1350956060811681800,1350956060811681800,1.610928e+12,2021-01-17 18:59:59,-0500,,La Policía Nacional alerta de un repunte de es...,es,[],[],...,,,,,,[],,,,
1,1350956017841016833,1350956017841016833,1.610928e+12,2021-01-17 18:59:48,-0500,,"Rhipe, NTT Data, Adobe win Microsoft APAC part...",en,[],[],...,,,,,,[],,,,
2,1350955927051132935,1350955921678151680,1.610928e+12,2021-01-17 18:59:27,-0500,,To Microsoft. See apple had no care about vide...,en,[],[],...,,,,,,[],,,,
3,1350955901428109318,1350917664055689218,1.610928e+12,2021-01-17 18:59:21,-0500,,@DeusAbutre777 Não encontrei os jogos dos estu...,pt,[],[],...,,,,,,"[{'screen_name': 'DeusAbutre777', 'name': 'Ber...",,,,
4,1350955897854570497,1350882238431436801,1.610928e+12,2021-01-17 18:59:20,-0500,,@SnoopyG7 Microsoft said its coming to gamepas...,en,[],[],...,,,,,,"[{'screen_name': 'SnoopyG7', 'name': 'Snoopy',...",,,,


In [33]:
# Filter on tweets only in english
stock_df = stock_df[stock_df['language'] == 'en']

# Pull in only the raw Twitter text
stock_df = pd.DataFrame(stock_df['tweet'])
stock_df = stock_df.reset_index(drop=True)

In [34]:
# Visualize New DataFrame
stock_df.head()

,tweet
0,"Rhipe, NTT Data, Adobe win Microsoft APAC part..."
1,To Microsoft. See apple had no care about vide...
2,@SnoopyG7 Microsoft said its coming to gamepas...
3,"How is it 2021, the year of constant zoom call..."
4,The chromium-based version of Microsoft Edge i...


# Sentiment Analysis

In [35]:
# Import nlkt vader and initialize a sentiment analyzer to be used on the raw twitter data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
analyzer = SentimentIntensityAnalyzer()
tweet_sent = {'Compound':[],
                 'Negative':[],
                 'Neutral':[],
                 'Positive':[]}
# Get sentiment from articles
for index, row in stock_df.iterrows():
    try:
        tweet_sentiment = analyzer.polarity_scores(row["tweet"])
        tweet_sent["Compound"].append(tweet_sentiment["compound"])
        tweet_sent["Negative"].append(tweet_sentiment["neg"])
        tweet_sent["Neutral"].append(tweet_sentiment["neu"])
        tweet_sent["Positive"].append(tweet_sentiment["pos"])
    except AttributeError:
        pass
# Add the newly created scores to the previously created bitcoin dataframe
tweet_sent_df = pd.DataFrame(tweet_sent)
stock_sentiment = tweet_sent_df.join(stock_df)
stock_sentiment.head()

,Compound,Negative,Neutral,Positive,tweet
0,0.8074,0.00,0.711,0.289,"Rhipe, NTT Data, Adobe win Microsoft APAC part..."
1,0.1245,0.07,0.852,0.078,To Microsoft. See apple had no care about vide...
2,-0.1957,0.12,0.880,0.000,@SnoopyG7 Microsoft said its coming to gamepas...
3,0.0000,0.00,1.000,0.000,"How is it 2021, the year of constant zoom call..."
4,0.7269,0.00,0.534,0.466,The chromium-based version of Microsoft Edge i...


In [40]:
# Split the DataFrame 
number_of_rows = len(stock_sentiment.in

TypeError: '(slice(None, 553.6, None), slice(None, None, None))' is an invalid key

In [ ]:
# Clean the Text Function
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')


def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
 # text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [ ]:
# Apply the clean text function above
stock['tweet'] = stock['tweet'].astype(str)
stock['tweet'] = stock['tweet'].apply(clean_text)
stock['tweet'] = stock['tweet'].str.replace('\d+', '')
stock.head()